This notebook demonstrates the use of LangChain and LangGraph libraries to create a chatbot capable of querying stock prices and performing workflow automation. It integrates tools, state graphs, and a language model to process user queries and generate responses.


### 1. **Imports and Setup**
    - Libraries such as `langchain`, `langgraph`, and `dotenv` are imported to enable chatbot functionality, tool integration, and environment variable management.
    - The `load_dotenv()` function is used to load environment variables from a `.env` file.

### 2. **State Definition**
    - A `State` class is defined using `TypedDict` to structure the chatbot's state, including messages processed by the graph.

### 3. **Tool Integration**
    - A tool named `get_stock_price` is implemented to fetch stock prices for given symbols (e.g., `MSFT`, `AAPL`, `AMZN`).
    - The tool is bound to the language model (`llm`) using `llm.bind_tools()`.

### 4. **State Graph Construction**
    - A `StateGraph` is built to define the workflow of the chatbot.
    - Nodes and edges are added to the graph to represent the flow of messages and tool invocations.
    - Conditional edges are used to determine when tools should be invoked.

### 5. **Graph Compilation**
    - The graph is compiled into a `CompiledStateGraph` object, which can process user queries and generate responses.

### 6. **Visualization**
    - The graph structure is visualized using Mermaid diagrams, rendered as PNG images.

### 7. **Query Execution**
    - User queries are passed to the graph using the `invoke()` method.
    - The chatbot processes the queries, invokes tools as needed, and returns responses.

---

## Variables in the Notebook

### Key Variables
- **`ChatGroq`**: Represents the language model used for chatbot functionality.
- **`START` and `END`**: Constants defining the start and end nodes of the graph.
- **`State`**: TypedDict defining the structure of the chatbot's state.
- **`ToolNode`**: Represents a node in the graph for tool invocation.
- **`builder`**: StateGraph object used to construct the workflow.
- **`get_stock_price`**: Tool for fetching stock prices.
- **`graph`**: CompiledStateGraph object for executing queries.
- **`llm` and `llm_with_tools`**: Language model and its binding with tools.
- **`msg`**: Example user query about stock purchases.
- **`state`**: Dictionary representing the chatbot's state after processing queries.
- **`tools`**: List of tools integrated into the chatbot.

---

## Example Queries
1. **Stock Price Query**:
    - User asks for the price of `AAPL` stock.
    - The chatbot invokes the `get_stock_price` tool and returns the price.

2. **Historical Inquiry**:
    - User asks about the inventor of the theory of relativity.
    - The chatbot processes the query and provides the name.

3. **Cost Calculation**:
    - User asks for the total cost of purchasing stocks.
    - The chatbot calculates the cost using current stock prices and returns the result.

In [ ]:
from langchain_groq import ChatGroq 
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
@tool
def get_stock_price(symbol: str) -> float:
    """
    Get the stock price for a given symbol.

    Args:
        symbol (str): The stock symbol (e.g., 'MSFT', 'AAPL', 'AMZN').

    Returns:
        float: The stock price for the given symbol, or 0.0 if the symbol is not found.
    """
    return {
        "MSFT": 495.94,
        "AAPL": 201.08,
        "AMZN": 223.3
    }.get(symbol, 0.0)

tools = [get_stock_price]

llm = ChatGroq(
        model="meta-llama/llama-4-maverick-17b-128e-instruct", 
        temperature=0.7
    )
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node(chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)

graph = builder.compile()

In [ ]:
from IPython import display
display.display(display.Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
state = graph.invoke({"messages": [{"role": "user", "content": "What is the price of AAPL stock right now?"}]})
print(state["messages"][-1].content)

In [ ]:
state = graph.invoke({"messages": [{"role": "user", "content": "Who invented theory of relativity? print person name only"}]})
print(state["messages"][-1].content)

In [19]:
msg = "I want to buy 20 AMZN stocks using current price. Then 15 MSFT. What will be the total cost?"

state = graph.invoke({"messages": [{"role": "user", "content": msg}]})
print(state["messages"][-1].content)

495.94
